In [1]:
import xml.etree.ElementTree as ET
tree = ET.parse('gpx/12043287.gpx')
root = tree.getroot()
root

<Element '{http://www.topografix.com/GPX/1/1}gpx' at 0x107a33010>

In [2]:
root.tag

'{http://www.topografix.com/GPX/1/1}gpx'

In [3]:
for child in root:
    print(child.tag, child.attrib)

{http://www.topografix.com/GPX/1/1}metadata {}
{http://www.topografix.com/GPX/1/1}trk {}


In [4]:
# Define the GPX namespace
ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}
element = root.find('.//gpx:trkpt', ns)
element

<Element '{http://www.topografix.com/GPX/1/1}trkpt' at 0x107a58900>

In [5]:
import pandas as pd
import xml.etree.ElementTree as ET  # For the alternative method

In [6]:
# Parse the GPX file
tree = ET.parse('gpx/12043287.gpx')
root = tree.getroot()

# Define the GPX namespace
ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}

# Extract data into a list of dictionaries
data = []
for trkpt in root.findall('.//gpx:trkpt', ns):
    lat = float(trkpt.attrib.get('lat'))  # Get lat attribute
    lon = float(trkpt.attrib.get('lon'))  # Get lon attribute
    time_elem = trkpt.find('gpx:time', ns)
    datetime_str = time_elem.text if time_elem is not None else None
    speed_elem = trkpt.find('gpx:extensions/gpx:speed', ns)
    speed = float(speed_elem.text) if speed_elem is not None else None
    data.append({'datetime': datetime_str, 'latitude': lat, 'longitude': lon, 'speed_mps': speed})
    break
print(data)

[{'datetime': '2025-08-17T07:03:22Z', 'latitude': 27.63719687, 'longitude': 85.3332576, 'speed_mps': 0.0}]


In [7]:
# Create DataFrame
df = pd.DataFrame(data)

# Convert 'datetime' to Pandas datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df['speed_kmph'] = df['speed_mps']*3.6
# Preview
print(df.head())

                   datetime   latitude  longitude  speed_mps  speed_kmph
0 2025-08-17 07:03:22+00:00  27.637197  85.333258        0.0         0.0


In [8]:
!pip install folium

In [9]:
import folium

# Assume your dataframe is called df
# df columns: datetime, latitude, longitude, speed_mps

# Center map on first point
m = folium.Map(
    location=[df.latitude.iloc[0], df.longitude.iloc[0]],
    zoom_start=15,
    tiles="OpenStreetMap"
)

# Create list of (lat, lon)
coords = list(zip(df.latitude, df.longitude))

# Add track
folium.PolyLine(
    coords,
    color="blue",
    weight=4,
    opacity=0.8
).add_to(m)

# Save map
m.save("gpx_track.html")

In [10]:
from branca.colormap import LinearColormap

# Create colormap
colormap = LinearColormap(
    colors=["blue", "yellow", "red"],
    vmin=df.speed_kmph.min(),
    vmax=df.speed_kmph.max(),
    caption="Speed (km/h)"
)

# Draw line segments with speed color
for i in range(len(df) - 1):
    speed = df.speed_kmph.iloc[i]
    folium.PolyLine(
        locations=[
            (df.latitude.iloc[i], df.longitude.iloc[i]),
            (df.latitude.iloc[i+1], df.longitude.iloc[i+1])
        ],
        color=colormap(speed),
        weight=5,
        opacity=0.9,
        popup=f"""
        Time: {df.datetime.iloc[i]}<br>
        Speed: {df.speed_kmph.iloc[i]:.2f} km/hs
        """
    ).add_to(m)

colormap.add_to(m)
m

In [11]:
import sqlite3

from bs4 import BeautifulSoup

def setup_db():
    conn = sqlite3.connect('osm_traces.db')
    curr = conn.cursor()
    curr.execute('''CREATE TABLE IF NOT EXISTS traces 
                    (id INTEGER PRIMARY KEY, tag TEXT, name TEXT)''')
    curr.execute('''CREATE TABLE IF NOT EXISTS points 
                    (trace_id INTEGER, lat REAL, lon REAL, 
                     time TEXT, speed REAL,
                     FOREIGN KEY(trace_id) REFERENCES traces(id))''')
    # --- CLEANUP STEP ---
    # Clear existing data so each run starts fresh
    print("Cleaning up old database records...")
    curr.execute("DELETE FROM points")
    curr.execute("DELETE FROM traces")
    
    conn.commit()
    return conn

In [12]:
import requests

tag = 'Melbourne'
url = f"https://www.openstreetmap.org/traces/tag/{tag}"

# Many servers block the default 'python-requests' User-Agent.
headers = {'User-Agent': 'Mozilla/5.0'} 

response = requests.get(url, headers=headers)

print(f"Status Code: {response.status_code}")
print(response.text[:500]) # Peek at the HTML

Status Code: 200
<!DOCTYPE html>
<html lang="en" dir="ltr">
  <head data-locale="en" data-preferred-editor="id" data-preferred-languages="[]">
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="apple-touch-icon" type="image/png" href="/assets/apple-touch-icon-57x57-0af2b52ac2b1a8cb0aba42f17aa901bfdedb9174bd978f838e3b3e1dd8602f52.png" sizes="57x57" />
<link rel="apple-touch-icon" type="image/png" href="/assets/apple-touch-icon-


In [13]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')

rows = soup.select('table#trace_list tbody tr')

print(f"Number of traces found on page: {len(rows)}")
# Print the first row to see the raw content
if rows:
    print(rows[0].get_text(separator=' | '))

Number of traces found on page: 20

 | 
 | 
 | 
 | 
 | 
 | 
 | 2025_09_19_12_21_Fri.gpx | 
 | 
 | 
          2865 points
         | 
 | 
 | PUBLIC | 
 | 
 | 
 | 
 | 4 months ago |  by  | jfd553 |  in  | Melbourne | 
 | 
 | 
      Car ride
     | 
 | 
 | 
 | 
 | 
 | 
 | View Map | 
 | 
 | 
 | Edit Map | 
 | 
 | 
 | 
 | 



In [14]:
import re

# We will test on a dummy string first
test_row_text = "by user123 (2,450 points) 2 minutes ago"

point_pattern = re.compile(r'([\d,]+)\s+points')

found_points = point_pattern.findall(test_row_text)
clean_number = found_points[0].replace(',', '')
point_count = int(clean_number)

print(f"Successfully converted: {point_count}")

Successfully converted: 2450


In [15]:
# Assume 'row' is the first valid row from our list
row = rows[0]

link_tag = row.select_one('a[href*="/traces/"]')

if link_tag:
    trace_id = link_tag['href'].split('/')[-1]
    
    trace_name = link_tag.get_text().strip()
    
    print(f"ID: {trace_id}, Name: {trace_name}")

ID: 12072620, Name: 


In [16]:
import requests
from bs4 import BeautifulSoup
import time
import re
def list_qualifying_traces(tag, target_count=5, min_points=2000):
    base_url = "https://www.openstreetmap.org"
    # Note: Using the tag search URL
    search_url = f"{base_url}/traces/tag/{tag}"

    point_pattern = re.compile(r'([\d,]+)\s+points')
    # Standard browser headers to prevent being throttled
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    found = 0
    page = 1

    print(f"--- Fast Search: {tag} (Min: {min_points} pts) ---")

    while found < target_count:
        print(f"Checking Page {page}...")
        try:
            response = requests.get(f"{search_url}/page/{page}", headers=headers, timeout=10)
            if response.status_code != 200:
                break
        except requests.exceptions.Timeout:
            print("Server took too long to respond. Retrying...")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # OSM trace rows usually have a specific structure
        # We look for rows that contain a link and the text 'points'
        rows = soup.select('table#trace_list tbody tr')
        if not rows:
            print("No traces found on this page.")
            break

        for row in rows:
            if found >= target_count:
                break

            # Optimization: Find the link and the point count directly
            # The point count is typically in a text node within a <td>
            row_text = row.get_text()
            # print(row)
            if 'points' not in row_text:
                continue
            # print(row_text)
                
            # Efficiently extract point number
            try:
                # Extracts the number before the word 'points'
                # e.g., " (2,450 points) " -> 2450
                found_points = point_pattern.findall(row_text)
                point_count = int(found_points[0].replace(',', ''))
            except (ValueError, IndexError):
                continue

            if point_count >= min_points:
                link_tag = row.select_one('a[href*="/traces/"]')
                if link_tag:
                    print(link_tag)
                    trace_name = link_tag.get_text().strip()
                    trace_id = link_tag['href'].split('/')[-1]
                    found += 1
                    print(f"SUCCESS: [{found}] ID: {trace_id} | {point_count} pts | {trace_name}")

        page += 1
        break
        # Small sleep to be a good webscrapping practice
        time.sleep(0.5)

    print(f"--- Found {found} traces total ---")

list_qualifying_traces('Melbourne', min_points = 200, target_count=5)

--- Fast Search: Melbourne (Min: 200 pts) ---
Checking Page 1...
<a class="d-inline-block" href="/user/jfd553/traces/12072620"><img alt="" class="trace_image" height="50" src="/user/jfd553/traces/12072620/icon" width="50"/></a>
SUCCESS: [1] ID: 12072620 | 2865 pts | 
<a class="d-inline-block" href="/user/jfd553/traces/12072619"><img alt="" class="trace_image" height="50" src="/user/jfd553/traces/12072619/icon" width="50"/></a>
SUCCESS: [2] ID: 12072619 | 987 pts | 
<a class="d-inline-block" href="/user/melbournefan/traces/3521343"><img alt="" class="trace_image" height="50" src="/user/melbournefan/traces/3521343/icon" width="50"/></a>
SUCCESS: [3] ID: 3521343 | 406 pts | 
<a class="d-inline-block" href="/user/melbournefan/traces/3508949"><img alt="" class="trace_image" height="50" src="/user/melbournefan/traces/3508949/icon" width="50"/></a>
SUCCESS: [4] ID: 3508949 | 992 pts | 
<a class="d-inline-block" href="/user/melbournefan/traces/3467303"><img alt="" class="trace_image" height="5

In [17]:
def scrape_osm_traces(tag, target_count=5, min_points=2000):
    """Phase 2 & 3: Scrape, Filter, and Download."""
    conn = setup_db()
    base_url = "https://www.openstreetmap.org"
    search_url = f"{base_url}/traces/tag/{tag}"
    headers = {'User-Agent': 'Mozilla/5.0 (Training Project)'}
    point_pattern = re.compile(r'([\d,]+)\s+points')
    
    found = 0
    page = 1

    print(f"--- Starting Scrape: {tag} (Min: {min_points} pts) ---")

    while found < target_count:
        print(f"Scanning Page {page}...")
        response = requests.get(f"{search_url}", headers=headers)
        if response.status_code != 200: break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('table#trace_list tbody tr')
        
        if not rows: break

        for row in rows:
            if found >= target_count: break

            row_text = row.get_text()
            if 'points' not in row_text: continue

            try:
                found_points = point_pattern.findall(row_text)
                point_count = int(found_points[0].replace(',', ''))
            except (ValueError, IndexError): continue

            if point_count >= min_points:
                # Targeted selector to get the text link, not the image link
                # This looks for the link that is NOT wrapping the image icon
                links = row.select('a[href*="/traces/"]')
                # Usually, the second link or the one with text is our target
                link_tag = next((l for l in links if l.get_text().strip()), None)
                
                if link_tag:
                    trace_name = link_tag.get_text().strip()
                    trace_id = link_tag['href'].split('/')[-1]
                    
                    print(f"Found: {trace_id} | {point_count} pts | {trace_name}")
                    
                    # Download the raw GPX data
                    download_url = f"{base_url}/trace/{trace_id}/data"
                    gpx_resp = requests.get(download_url, headers=headers)
                    
                    if gpx_resp.status_code == 200:
                        parse_gpx_and_save(conn, trace_id, tag, trace_name, gpx_resp.content)
                        found += 1
                        time.sleep(1) # Be a polite scraper
        # --- DYNAMIC PAGINATION LOGIC ---
        # Find the link that contains "Older Traces"
        next_page_link = soup.find('a', title="Older Traces")
        # print(next_page_link)
        if next_page_link and next_page_link.get('href'):
            # Update current_url for the next loop iteration
            search_url = base_url + next_page_link['href']         
            page += 1
        else:
            print("No 'Older Traces' link found. Reached the end of the results.")
            break        # Removing the 'break' here allows it to continue to page 2 if needed

    conn.close()
    print(f"--- Finished: {found} traces processed ---")

In [18]:
def parse_gpx_and_save(conn, trace_id, tag, name, gpx_xml):
    root = ET.fromstring(gpx_xml)
    
    # Namespaces are required for GPX
    ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}
    
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO traces VALUES (?, ?, ?)", 
                   (trace_id, tag, name))

    points_to_insert = []
    
    # Drill down: trk -> trkseg -> trkpt
    for trkpt in root.findall('.//gpx:trkpt', ns):
        lat = float(trkpt.attrib.get('lat'))
        lon = float(trkpt.attrib.get('lon'))
        
        # Get Time
        time_elem = trkpt.find('gpx:time', ns)
        dt = time_elem.text if time_elem is not None else None
        
        # Get Speed (sometimes nested in extensions)
        speed_elem = trkpt.find('.//gpx:speed', ns)
        speed = float(speed_elem.text) if speed_elem is not None else 0.0
        
        points_to_insert.append((trace_id, lat, lon, dt, speed))
    
    # Efficiently insert all points at once
    cursor.executemany("INSERT INTO points VALUES (?, ?, ?, ?, ?)", 
                       points_to_insert)
    conn.commit()
    print(f"Saved {len(points_to_insert)} points for Trace {trace_id}")

In [19]:
# Run the full process
scrape_osm_traces('Melbourne', target_count=5, min_points=2000)

Cleaning up old database records...
--- Starting Scrape: Melbourne (Min: 2000 pts) ---
Scanning Page 1...
Found: 12072620 | 2865 pts | 2025_09_19_12_21_Fri.gpx
Saved 2865 points for Trace 12072620
Found: 3358899 | 4430 pts | morning__loop__in_the_inner_suburbs_2020_07_03_11_19_56.gpx
Saved 4430 points for Trace 3358899
Found: 3345792 | 4430 pts | morning__loop__in_the_inner_suburbs_2020_07_03_11_19_56.gpx
Saved 4430 points for Trace 3345792
Found: 3342777 | 2069 pts | Train_Trip_on_a_Sunday_morning_2020_06_28_10_31_07.gpx
Saved 2069 points for Trace 3342777
Scanning Page 2...
Found: 3082959 | 4887 pts | Saturday_evening_drive_into_Melbourne_2019_08_24_17_04_58.gpx
Saved 4887 points for Trace 3082959
--- Finished: 5 traces processed ---


In [20]:
import pandas as pd
import sqlite3
import folium

# Connect to the existing database
conn = sqlite3.connect('osm_traces.db')

# Access the 'traces' table
df_traces = pd.read_sql_query("SELECT * FROM traces", conn)
print(f"Found {len(df_traces)} traces in the database.")

Found 5 traces in the database.


In [21]:
for index, row in df_traces.iterrows():
    trace_id = row['id']
    trace_name = row['name']
    
    # Read the points table for this specific trace
    query = f"SELECT * FROM points WHERE trace_id = {trace_id}"
    df_points = pd.read_sql_query(query, conn)
    print(f"Found {len(df_points)} points in trace:{trace_id}.")

Found 4887 points in trace:3082959.
Found 2069 points in trace:3342777.
Found 4430 points in trace:3345792.
Found 4430 points in trace:3358899.
Found 2865 points in trace:12072620.


In [22]:
# Initialize a global map centered on the first point of the first trace
# We use the first point available in the points table for initial centering
sample_point = pd.read_sql_query("SELECT lat, lon FROM points LIMIT 1", conn)

m = folium.Map(
    location=[sample_point.lat[0], sample_point.lon[0]],
    zoom_start=13,
    tiles="OpenStreetMap"
)

# Colors to differentiate different traces
colors = ['blue', 'red', 'green', 'orange', 'purple']

for index, row in df_traces.iterrows():
    trace_id = row['id']
    trace_name = row['name']
    
    # Read the points table for this specific trace
    query = f"SELECT * FROM points WHERE trace_id = {trace_id}"
    df_points = pd.read_sql_query(query, conn)
    
    if df_points.empty:
        continue

    # Exercise: Add kmph column (Conversion: 1 m/s = 3.6 km/h)
    df_points['speed_kmph'] = df_points['speed'] * 3.6
    
    # Convert points to a list of (lat, lon) tuples
    coords = list(zip(df_points.lat, df_points.lon))
    
    # Create a line for the trace and add it to the map
    # A popup is added so you can click a line to see its name
    folium.PolyLine(
        coords,
        color=colors[index % len(colors)],
        weight=4,
        opacity=0.7,
        popup=f"Trace: {trace_name} (ID: {trace_id})"
    ).add_to(m)

# Save the multi-trace map
# m.save("all_osm_traces.html")
# conn.close()
m